In [96]:
from PIL import Image
import numpy as np
from pathlib import Path

from core.utils import Picture

In [97]:
# Setting up I/O
input_dir = Path.cwd() / "data" / "raw"
output_dir = Path.cwd() / "data" / "output"


In [98]:
# image channelling
# red = img.copy()
# green = img.copy()
# blue = img.copy()

# red = img[:, :, 0]
# green = img[:, :, 1]
# blue = img[:, :, 2]
# channels = {
#     "red": red,
#     "green": green,
#     "blue": blue
# }

In [99]:
# new_image = img.copy()
# new_title = "Impulse Response Analysis" # IRA
# ira = rng.random_integers(0, 255, size=(height, width, channels))
# ira_mixed = (new_image.astype(float) + ira.astype(float)) / 2
# new_image[:] = ira_mixed.astype('uint8')
# # or
# new_image[:,::2] = ira[:, ::2]


In [100]:
# new_image = img.copy()
# new_title = "Эффект ненастроеного телевизора"
# new_image[:,:] = rng.random_integers(0, 255, size=(height, width, channels))

In [101]:
# new_image = img.copy()
# new_title = "10% noise"
# new_image[rng.random(size=(height, width)) < 0.1] = rng.random_integers(0, 256, (3,))

In [102]:
# We will try to flip the image horizontally
# new_image = img.copy()
# new_title = "horizontally flipped"
# new_image = new_image[:,::-1]

In [103]:
# We will try vertically flipped
# new_image = img.copy()
# new_title = "vertically flipped"
# new_image = new_image[::-1, :]

In [104]:
# We will try both flippes
# new_image = img.copy()
# new_title = "Horizontally and Verticallu flipped"
# new_image = new_image[::-1, ::-1]

In [105]:
# We will try crop center
# new_image = img.copy()
# new_title = "Cropped Image"
# min_y, max_y = np.round(height * 0.2).astype(int), np.round(height * 0.8).astype(int)
# min_x, max_x = np.round(width * 0.2).astype(int), np.round(width * 0.8).astype(int)
# new_image = new_image[min_y: max_y, min_x: max_x]

In [106]:
# We will try invert color
# new_image = img.copy()
# new_title = "Inverted Colors"
# new_image[:, :, :3] = 1.0 - new_image[:, :, :3]


# Forward mapping
1. We get original coord [y,x]
2. Calculate center, c_y, c_xw = height/2, width/2
3. Move original coord to center, moved_coord = orig[y,x] - [c_y, c_x]
4. Use following formulas to calculate turned coord
    x' = xcosa - ysina
    y' = xsina + ycosa
5. Move turned coord back to his place and get new coord for original coord:
    new_x = x' + c_x
    new_y = y' + c_y

# Inverse mapping formule
x = x'cosa + y'sina
y = -x'sina + y'cosa 

x' = xcosa - ysina 
y' = xsina + ycosa

new_x = x' + c_x
new_y = y' + c_y


old_x = x - c_x
old_y = y - c_x

x = old_x*cosa + old_y*sina
y = -old_x*sina + old_y*cosa 


# Inversive mapping with bilinear interpolation 

1. Create new empty matrix, so turned image fits in it.
2. For each pixel in new matrix we perform inverse mapping to determine its color in original image.

3. For every result of inversive mapping we perform bilinear interpolation to determine its color. 
    x0,y0 - the top left corner
    x1,y1 - the bottom right corner
    x0 = round(x)
    x1 = x0 + 1
    y0 = round(y)
    y1 = y0 + 1

    adjacent pixels
    Q00 = I(x0, y0)
    Q10 = I(x1, y0)
    Q01 = I(x0, y1)
    Q11 = I(x1, y1)

    dx = x - x0 (0 <= dx < 1)
    dy = y - y0 (0 <= dy < 1)

    Calculate weight of each adjecent pixel:
    Q00_w = (1 - dx)(1 - dy)
    Q10_w = dx(1 - dy)
    Q01_w = (1 - dx)dy
    Q11_w = dx*dy

    Formule of bilenear interpolation 
    I(x, y) = Q00_w + Q10_w + Q01_w + Q11_w

In [107]:
# # Размер холста
# cos_a, sin_a = np.cos(np.deg2rad(78)), np.sin(np.deg2rad(78))
# new_w = math.ceil(abs(width * cos_a) + abs(height * sin_a))
# new_h = math.ceil(abs(height * cos_a) + abs(width * sin_a))

# # Перенос центра
# cy_new, cx_new = new_h // 2, new_w // 2
# cy_old, cx_old = height // 2, width // 2

# Y0 = Y - cy_new
# X0 = X - cx_new

# # Inverse mapping
# X_old = X0 * cos_a + Y0 * sin_a + cx_old
# Y_old = -X0 * sin_a + Y0 * cos_a + cy_old

# # Bilinear interpolation
# x0, y0 = np.floor(X_old).astype('uint8'), np.floor(Y_old).astype('uint8')

# x0 = np.clip(x0, 0, width-1)
# y0 = np.clip(y0, 0, height-1)
# x1 = np.clip(x0 + 1, 0, width-1)
# y1 = np.clip(y0 + 1, 0, height-1)


# dx = X_old - x0
# dy = Y_old - y0

# w00 = (1-dx)*(1-dy)
# w01 = dx*(1-dy)
# w10 = (1-dx)*dy
# w11 = dx*dy

# out = (
#     new_image[y0,x0] * w00[..., None] +
#     new_image[y0,x1] * w01[..., None] +
#     new_image[y1,x0] * w10[..., None] +
#     new_image[y1,x1] * w11[..., None]
#     )

# mask = (X_old>=0)&(X_old<width)&(Y_old>=0)&(Y_old<height)
# out[~mask] = 0

# turned_box = np.zeros((new_h, new_w, 3), dtype='uint8')
# turned_box[mask] = out[mask].astype('uint8')
# new_title = "turned_box_02_05"

In [108]:
# # Генерация картинки на R*R пикселей
# r = 4
# random_box = np.random.random_integers(0, 256, size=(2*r+1, 2*r+1, 3))
# h, w, c = random_box.shape
# new_title = "before blur" 

In [109]:
# # Creating input image
# plt.figure()
# plt.imshow(random_box)
# plt.title(new_title)
# plt.axis('off')

# plt.savefig(f"{output_dir}/{new_title}.png", dpi = 300)
# plt.close()

In [110]:
# # Определяем веса каждого слоя сразу
# weights_by_layer = [
#     [1], # без уровня
#     [0.5, 0.5], # 1 слой
#     [0.5, 0.25, 0.25], # 2 слоя
#     [0.5, 0.25, 0.125, 0.125], # 3 слоя
#     [0.5, 0.25, 0.125, 0.0625, 0.0625] # 4 слоя
# ] # больше 4 слооев нет смысла считать из-за низкого влияния

# current_weights_by_layer = weights_by_layer[r]

In [111]:
# Создаем матрицу для складывания каналов по слойно
# sum_by_layer = np.zeros((r+1, 3)) # Сюда будем складывать значения каждого слоя
# sum_by_layer

In [112]:
# Получаем сумму всех каналов по слойно и их веса
# for y in range(h):
#     for x in range(w):
#         what_layer = max(abs(x-r), abs(y-r))
#         sum_by_layer[what_layer] += random_box[y, x] * current_weights_by_layer[what_layer]
# sum_by_layer

In [113]:
# Сумму нашли, теперь получаем среднее значение канала каждого слоя
# for i in range(1, r+1):
#     for j in range(3):
#         amount = (((2 * i) + 1) * 2) + ((((i - 1) * 2) + 1) * 2)
#         sum_by_layer[i][j] /= amount
# sum_by_layer

In [114]:
# Меняем цвет центрального квадрата по алгоритму. Вуаля)))
# edited_random_box = random_box
# print(edited_random_box[h // 2, w // 2] )
# edited_random_box[h // 2, w // 2] = sum_by_layer.sum(axis=0).astype('uint8')
# edited_random_box[h // 2, w // 2] 

In [115]:
# Creating Output image
# new_title = "after blur"

# plt.figure()
# plt.imshow(edited_random_box)
# plt.title(new_title)
# plt.axis('off')

# plt.savefig(f"{output_dir}/{new_title}.png", dpi = 300)
# plt.close()

In [116]:

# def aj_blur(bbox, r):
#     h, w, c = bbox.shape

#     weights_by_layer = [
#         [1.0],
#         [0.35, 0.35, 0.3],
#         [0.3, 0.25, 0.2, 0.15, 0.1],
#         [0.25, 0.22, 0.2, 0.18, 0.1, 0.05],
#         [0.2, 0.2, 0.18, 0.16, 0.14, 0.12, 0.1]
#     ]

#     current_weights = weights_by_layer[r]
#     sum_by_layer = np.zeros((r+1, 3))

#     for y in range(h):
#         for x in range(w):
#             layer = max(abs(x-r), abs(y-r))
#             sum_by_layer[layer] += bbox[y, x] * current_weights[layer]

#     # нормализация слоёв
#     for i in range(1, r+1):
#         sum_by_layer[i] /= (8 * i)

#     # центр
#     sum_by_layer[0] /= 1.0

#     # корректная глобальная нормализация
#     effective_weight_sum = sum(current_weights[:r+1])

#     result = sum_by_layer.sum(axis=0) / effective_weight_sum
#     return np.clip(result, 0, 255).astype(np.uint8)

# # from tqdm import tqdm

# r = 2
# h, w, c = img.shape
# new_img = img.copy()


#     for y in tqdm(range(r, h - r), desc="AJ blur"):
#         for x in range(r, w - r):
#             window = img[y - r:y + r + 1, x - r:x + r + 1]
#             new_img[y, x] = aj_blur(window, r)

# # Creating input image
# new_title = "AJ Triple Super Very hard Blur algorithm"
# plt.figure()
# plt.imshow(new_img)
# plt.title(new_title)
# plt.axis('off')

# plt.savefig(f"{output_dir}/{new_title}.jpg", dpi = 300)
# plt.close()

Сжатие
Шаг №1. image input

In [200]:

choose_pic = Picture.GUY
inp = Image.open(f"{input_dir}/{choose_pic.value}")
img = np.array(inp, dtype='uint16')
h, w, c = img.shape
a = 224
print(f"height is: {h}, height // a: {h // a}, we need to erase: {h - (h // a) * a} rows")
print(f"width is: {w}, width // a: {w // a}, we need to erase: {w - (w // a) * a} cols")

height is: 500, height // a: 2, we need to erase: 52 rows
width is: 500, width // a: 2, we need to erase: 52 cols


Шаг №2. Изменение img.shape для кратности нужному формату (224, 256)

Можно сделать так:
- Определяем индексы столбцов на удаление -> создаем список "cols_to_del"
- Проходим по каждому столбцу:
    - чтобы не проходить по каждому пикселю сделаем перемножение сетки на скаляр
    - берем сетку из 3 тензоров, left center right в каждом axis
    - у left & right уменьшаем каналы, перемножив на 0.5
    - затем у left & right увеличиваем каналы, добавив 0.5 * каналы удаляемого пикселя
    - удаляем cols then rows


In [201]:
def reduction_to_multiplicity(img: np.ndarray, a: int) -> np.ndarray:
    h, w, c = img.shape
    src = img.astype(np.float32)
    dst = src.copy()

    row_amount = h - (h // a) * a
    col_amount = w - (w // a) * a

    remove_col_idx = np.linspace(1, w - 2, col_amount, dtype=int)
    remove_row_idx = np.linspace(1, h - 2, row_amount, dtype=int)


    for j in remove_col_idx: 
        center = src[:, j : j+1]

        dst[:, j-1 : j] *= 0.5
        dst[:, j+1 : j+2] *= 0.5

        dst[:, j-1 : j] += center * 0.5
        dst[:, j+1 : j+2] += center * 0.5
    dst = np.delete(dst, remove_col_idx, axis=1)
    src = dst.copy()

    for i in remove_row_idx:
        center = src[i : i+1, :]

        dst[i-1 : i, :] *= 0.5
        dst[i+1 : i+2, :] *= 0.5

        dst[i-1 : i, :] += center * 0.5
        dst[i+1 : i+2, :] += center * 0.5

    dst = np.delete(dst, remove_row_idx, axis=0)
    dst = np.clip(dst, 0, 255)


    return dst

img = reduction_to_multiplicity(img, a)
img.shape

(448, 448, 3)

In [202]:
nimg = np.zeros((a, a, 3), dtype=np.float32)
nh, nw, _ = nimg.shape

gh = h // nh
gw = w // nw

nimg = img.reshape(
    a, gh,
    a, gw,
    3
).mean(axis=(1, 3))


# for r in range(nh):
#     for c in range(nw):
#         r0, r1 = r * gh, (r + 1) * gh
#         c0, c1 = c * gw, (c + 1) * gw

#         nimg[r, c] = img[r0 : r1, c0 : c1].mean(axis=(0, 1))

nimg = np.clip(nimg, 0, 255).astype(np.uint8)


In [203]:
print(nimg.shape)

Image.fromarray(nimg).save(f"{output_dir}/{choose_pic.value} - after.png")


(224, 224, 3)
